# Time-based Features and Moving Averages
### CNN-LSTM using All-in-one Approach (7.1 from report)

Anastasios Koutsogiannopoulos, 03321

Fani Banou, 03322

Ioannis Sakellariou, 03144

In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, BatchNormalization, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error

### Data Loading and Preprocessing Functions

In [ ]:
def mount_drive():
    """
    Mount Google Drive if running on Google Colab.
    """
    try:
        from google.colab import drive
        drive.mount('/content/drive/')
    except ImportError:
        print("Not running on Google Colab; skipping drive mount.")


In [ ]:
def load_and_preprocess_data(file_path):
    """
    Load the dataset from the CSV file, convert UNIX timestamps to datetime,
    set the datetime as index, and resample the data to hourly frequency.
    """
    # Load dataset
    data = pd.read_csv(file_path)

    # Convert 'unix' column to datetime and set as index
    data['date'] = pd.to_datetime(data['unix'], unit='s')
    data.set_index('date', inplace=True)
    data.sort_index(ascending=True, inplace=True)

    # Use only the 'close' column
    numeric_data = data[['close']].copy()

    # Resample data to hourly frequency in case the data is in minutes
    data_hourly = numeric_data.resample('1h').mean()

    # Add time-based features and cyclic features
    data_hourly = add_time_features(data_hourly)
    data_hourly = add_cyclic_features(data_hourly)

    # Compute Moving Averages
    data_hourly['MA_24'] = data_hourly['close'].rolling(window=24).mean()
    data_hourly['MA_240'] = data_hourly['close'].rolling(window=240).mean()

    # Drop rows with NaN values due to rolling windows
    data_hourly.dropna(inplace=True)

    return data_hourly

In [ ]:
def add_time_features(df):
    """
    Add time-based features hour, day, weekday, month extracted from the index.
    """
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['weekday'] = df.index.weekday
    df['month'] = df.index.month
    return df

In [ ]:
def add_cyclic_features(df):
    """
    Add cyclic sine and cosine representations for hour, weekday, and month.
    """
    # Cyclic features for hour
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    # Cyclic features for weekday
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    # Cyclic features for month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    return df

In [ ]:
def create_final_dataset(data_hourly, feature_columns, target_column):
    """
    Create the final dataset containing selected features and the target.
    """
    return data_hourly[feature_columns + [target_column]]

In [ ]:
def split_dataset(data, train_end_date, test_start_date, test_end_date):
    """
    Split the dataset into training and testing sets based on specified date ranges.
    """
    train_data = data[:train_end_date]
    test_data = data[test_start_date:test_end_date]
    return train_data, test_data

In [ ]:
def scale_data(train_data, test_data):
    """
    Scale the training and testing data using MinMaxScaler.
    """
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)
    return train_scaled, test_scaled, scaler

### Sequence Generation Functions

In [ ]:
def create_sequences_multi_step(data, seq_length, horizon):
    """
    Create input sequences and corresponding multi-step target sequences.
    """
    X, y = [], []
    # Create samples only until there are enough steps for the forecast horizon
    for i in range(len(data) - seq_length - horizon + 1):
        X.append(data[i:i+seq_length, :-1])
        y.append(data[i+seq_length:i+seq_length+horizon, -1])
    return np.array(X), np.array(y)

### Model Building and Training Functions

In [ ]:
def build_multi_step_model(sequence_length, num_features, forecast_horizon):
    """
    Build a Sequential model for direct multi-step forecasting.
    The model consists of convolutional layers, an LSTM block, and dense layers.
    """
    model = Sequential([
        # Convolutional Block
        Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, num_features)),
        BatchNormalization(),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Dropout(0.5),

        # LSTM Block: Stacked LSTM layers to capture temporal dependencies
        LSTM(256, return_sequences=True, input_shape=(sequence_length, num_features)),
        BatchNormalization(),
        Dropout(0.3),
        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.2),


        # Dense Block: Process the extracted features
        Dense(64, activation='relu'),
        Dropout(0.2),

        # Final Dense Layer to produce the multi-step forecast
        Dense(forecast_horizon, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])
    return model


In [ ]:
def train_model(model, X_train, y_train, epochs=100, batch_size=256, validation_split=0.2, patience=15):
    """
    Train the model using the given training data and early stopping to prevent overfitting.
    """
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=validation_split,
        shuffle=False,
        callbacks=[early_stopping]
    )
    return history

In [ ]:
def plot_loss_history(history):
    """
    Plot the evolution of training and validation loss over epochs.
    """
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.title('Loss Evolution During Training')
    plt.show()

### Forecasting and Evaluation Functions

In [ ]:
def make_forecast(model, train_scaled, sequence_length, num_features, scaler, feature_columns, forecast_horizon):
    """
    Make a multi-step forecast using the last available sequence from the training data.

    The model outputs scaled predictions which are then converted back to the original scale
    using the scaler's inverse_transform.
    """
    # Prepare input sequence for prediction features only
    X_input = train_scaled[-sequence_length:, :-1].reshape(1, sequence_length, num_features)

    # Predict the multi-step forecast output shape: 1, forecast_horizon
    forecast_scaled = model.predict(X_input)
    forecast_scaled = forecast_scaled.flatten()

    # Create a dummy array for the feature columns to use with the scaler's inverse_transform
    dummy = np.zeros((forecast_scaled.shape[0], len(feature_columns)))

    # Concatenate dummy features with the forecast target values to create a complete vector
    forecast_concat = np.concatenate([dummy, forecast_scaled.reshape(-1, 1)], axis=1)

    # Inverse transform to retrieve the forecast in the original scale extract the target column
    forecast_unscaled = scaler.inverse_transform(forecast_concat)[:, -1]
    return forecast_unscaled

In [ ]:
def plot_forecast(test_data, target_column, forecast_unscaled, forecast_horizon):
    """
    Plot the actual target values versus the predicted forecast.
    """
    test_index = test_data.index[:forecast_horizon]
    plt.figure(figsize=(12, 6))
    plt.plot(test_index, test_data[target_column].values[:forecast_horizon],
             label="Actual Close", color='blue', linestyle='dashed')
    plt.plot(test_index, forecast_unscaled, label="Predicted Close", color='red')
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.title("Actual vs Predicted Close Prices (Direct Multi-Step Forecasting)")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def evaluate_forecast(actual, forecast):
    """
    Evaluate the forecast using common regression metrics and print the results.
    """

    mae = mean_absolute_error(actual, forecast)
    mse = mean_squared_error(actual, forecast)
    rmse = np.sqrt(mse)

    print("Mean Absolute Error (MAE):", mae)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)

In [ ]:
def run_multiple_experiments_multi_step(num_runs, X_train, y_train, train_scaled, sequence_length,
                                        num_features, scaler, feature_columns, forecast_horizon,
                                        test_data, target_column):
    """
    Run the multi-step forecasting experiment multiple times num_runs,
    and average the training histories, forecasts, and evaluation metrics.
    """
    training_histories = []  # To store training histories for each run
    forecasts = []           # To store forecast predictions for each run
    metrics_list = []        # To store evaluation metrics for each run
    train_times = []         # To store execution times for model training
    forecast_times = []      # To store execution times for forecasting

    # Actual target values from test data for forecast_horizon
    actual = test_data[target_column].values[:forecast_horizon]

    for run in range(num_runs):
        print(f"Run {run+1}/{num_runs}")
        # Build the multi-step forecasting model
        model = build_multi_step_model(sequence_length, num_features, forecast_horizon)

        # Train the model
        start_time = time.time()
        history = train_model(model, X_train, y_train)
        train_time = time.time() - start_time
        train_times.append(train_time)

        training_histories.append(history)

        # Make forecast using the last sequence from the training data
        start_time = time.time()
        forecast_unscaled = make_forecast(model, train_scaled, sequence_length, num_features, scaler, feature_columns, forecast_horizon)
        forecast_time = time.time() - start_time
        forecast_times.append(forecast_time)

        forecasts.append(forecast_unscaled)

        # Compute evaluation metrics for this run
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        mae = mean_absolute_error(actual, forecast_unscaled)
        mse = mean_squared_error(actual, forecast_unscaled)
        rmse = np.sqrt(mse)
        metrics_list.append({'MAE': mae, 'MSE': mse, 'RMSE': rmse})

    # Determine the minimum number of epochs across runs due to early stopping
    min_epochs = min(len(hist.history['loss']) for hist in training_histories)
    # Average the training and validation loss curves over the minimum number of epochs
    avg_loss = np.mean([hist.history['loss'][:min_epochs] for hist in training_histories], axis=0)
    avg_val_loss = np.mean([hist.history['val_loss'][:min_epochs] for hist in training_histories], axis=0)

    # Plot average training history
    plt.figure(figsize=(10, 5))
    epochs_range = range(1, min_epochs+1)
    plt.plot(epochs_range, avg_loss, label='Average Training Loss')
    plt.plot(epochs_range, avg_val_loss, label='Average Validation Loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss (MSE)")
    plt.title(f"Average Loss Evolution over {num_runs} Runs")
    plt.legend()
    plt.show()

    # Average the forecast predictions
    avg_forecast = np.mean(np.array(forecasts), axis=0)

    # Plot actual vs. average forecast
    test_index = test_data.index[:forecast_horizon]
    plt.figure(figsize=(12, 6))
    plt.plot(test_index, actual, label="Actual Close", color='blue', linestyle='dashed')
    plt.plot(test_index, avg_forecast, label="Average Predicted Close", color='red')
    plt.xlabel("Date")
    plt.ylabel("Close Price")
    plt.title("Actual vs Average Predicted Close Prices (Multi-Step Forecasting)")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Average evaluation metrics over all runs
    avg_metrics = {
         'MAE': np.mean([m['MAE'] for m in metrics_list]),
         'MSE': np.mean([m['MSE'] for m in metrics_list]),
         'RMSE': np.mean([m['RMSE'] for m in metrics_list]),
    }
    print(f"Average Metrics over {num_runs} Runs:")
    print("Mean Absolute Error (MAE):", avg_metrics['MAE'])
    print("Mean Squared Error (MSE):", avg_metrics['MSE'])
    print("Root Mean Squared Error (RMSE):", avg_metrics['RMSE'])

    # Compute and print average execution times
    avg_train_time = np.mean(train_times)
    avg_forecast_time = np.mean(forecast_times)
    print(f"\nAverage Execution Times over {num_runs} Runs:")
    print(f"Model Training Time: {avg_train_time:.4f} seconds")
    print(f"Forecasting Time: {avg_forecast_time:.4f} seconds")


### Main Function

In [ ]:
def main():
    # Mount Google Drive
    mount_drive()

    # Define the file path for the dataset
    file_path = "/content/drive/MyDrive/bitcoin-dataset.csv"

    # Load and preprocess the dataset
    data_hourly = load_and_preprocess_data(file_path)

    # Define the feature columns and target column
    feature_columns = ['MA_24', 'MA_240',
                       'hour_sin', 'hour_cos',
                       'weekday_sin', 'weekday_cos',
                       'month_sin', 'month_cos']
    target_column = 'close'

    # Create the final dataset with the selected features and target
    data_final = create_final_dataset(data_hourly, feature_columns, target_column)

    # Define date ranges for the training and testing sets
    train_end_date = '2022-02-18 23:59:59'
    test_start_date = '2022-02-19 00:00:00'
    test_end_date = '2022-02-28 23:59:59'

    # Split the data into training and testing sets
    train_data, test_data = split_dataset(data_final, train_end_date, test_start_date, test_end_date)

    # Scale the data using MinMaxScaler
    train_scaled, test_scaled, scaler = scale_data(train_data, test_data)

    # Set the sequence length and forecast horizon, forecast horizon equals the number of rows in the test set
    sequence_length = 120
    forecast_horizon = test_scaled.shape[0]

    # Create multi-step sequences from the training data
    X_train, y_train = create_sequences_multi_step(train_scaled, sequence_length, forecast_horizon)

    num_features = X_train.shape[2]

    # # Build the multi-step forecasting model
    # model = build_multi_step_model(sequence_length, num_features, forecast_horizon)
    # model.summary()

    # # Train the model
    # history = train_model(model, X_train, y_train)

    # # Plot the evolution of training and validation loss
    # plot_loss_history(history)

    # # Make a multi-step forecast using the last available sequence from training data
    # forecast_unscaled = make_forecast(model, train_scaled, sequence_length, num_features, scaler, feature_columns, forecast_horizon)

    # # Plot actual vs. predicted close prices
    # plot_forecast(test_data, target_column, forecast_unscaled, forecast_horizon)

    # # Evaluate the forecast using regression metrics
    # actual = test_data[target_column].values[:forecast_unscaled.shape[0]]
    # evaluate_forecast(actual, forecast_unscaled)

    print("\nRunning multiple experiments...")
    run_multiple_experiments_multi_step(num_runs=10, X_train=X_train, y_train=y_train, train_scaled=train_scaled,
                                        sequence_length=sequence_length, num_features=num_features,
                                        scaler=scaler, feature_columns=feature_columns,
                                        forecast_horizon=forecast_horizon, test_data=test_data,
                                        target_column=target_column)


if __name__ == "__main__":
    main()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).

Running multiple experiments...
Run 1/10
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.3806 - mean_absolute_error: 0.4950 - val_loss: 0.1440 - val_mean_absolute_error: 0.3516
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1473 - mean_absolute_error: 0.3114 - val_loss: 0.1003 - val_mean_absolute_error: 0.2819
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1321 - mean_absolute_error: 0.2953 - val_loss: 0.0860 - val_mean_absolute_error: 0.2555
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1183 - mean_absolute_error: 0.2808 - val_loss: 0.0675 - val_mean_absolute_error: 0.2201
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.1135 - mean_absolute_error: 0.2746 - val_loss: 0.0637 - val_mean_absolute_error: 0.2120
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.1060 - mean_absolute_error: 0.2663 - val_loss: 0.0503 - val_mean_absolute_error: 0.1812
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0979 - mean_absolute_error: 0.2569 - val_loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 0.3436 - mean_absolute_error: 0.4711 - val_loss: 0.1270 - val_mean_absolute_error: 0.3253
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1435 - mean_absolute_error: 0.3073 - val_loss: 0.0941 - val_mean_absolute_error: 0.2686
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1348 - mean_absolute_error: 0.2980 - val_loss: 0.0454 - val_mean_absolute_error: 0.1740
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1227 - mean_absolute_error: 0.2838 - val_loss: 0.0707 - val_mean_absolute_error: 0.2254
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1064 - mean_absolute_error: 0.2659 - val_loss: 0.0749 - val_mean_absolute_error: 0.2339
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0901 - mean_absolute_error: 0.2478 - val_loss: 0.0739 - val_mean_absolute_error: 0.2321
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0762 - mean_absolute_error: 0.2265 - v

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 42ms/step - loss: 0.3463 - mean_absolute_error: 0.4743 - val_loss: 0.1401 - val_mean_absolute_error: 0.3452
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - loss: 0.1457 - mean_absolute_error: 0.3096 - val_loss: 0.1070 - val_mean_absolute_error: 0.2922
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.1309 - mean_absolute_error: 0.2938 - val_loss: 0.0818 - val_mean_absolute_error: 0.2472
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1196 - mean_absolute_error: 0.2824 - val_loss: 0.0668 - val_mean_absolute_error: 0.2156
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1173 - mean_absolute_error: 0.2804 - val_loss: 0.0575 - val_mean_absolute_error: 0.1971
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.1108 - mean_absolute_error: 0.2727 - val_loss: 0.0516 - val_mean_absolute_error: 0.1851
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1008 - mean_absolute_error: 0.2603 - val_loss: 0.0

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 48ms/step - loss: 0.3675 - mean_absolute_error: 0.4874 - val_loss: 0.1373 - val_mean_absolute_error: 0.3409
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1486 - mean_absolute_error: 0.3132 - val_loss: 0.0799 - val_mean_absolute_error: 0.2431
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1284 - mean_absolute_error: 0.2917 - val_loss: 0.0538 - val_mean_absolute_error: 0.1899
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1209 - mean_absolute_error: 0.2838 - val_loss: 0.0420 - val_mean_absolute_error: 0.1625
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.1150 - mean_absolute_error: 0.2771 - val_loss: 0.0421 - val_mean_absolute_error: 0.1631
Epoch 6/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.1098 - mean_absolute_error: 0.2724 - val_loss: 0.0330 - val_mean_absolute_error: 0.1457
Epoch 7/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.1017 - mean_absolute_error: 0.2628 - v

Average Metrics over 10 Runs:
Mean Absolute Error (MAE): 3769.850524391528
Mean Squared Error (MSE): 22229227.117580656
Root Mean Squared Error (RMSE): 4053.8296815674194

Average Execution Times over 10 Runs:
Model Training Time: 46.5175 seconds
Forecasting Time: 0.4304 seconds
